In [1]:
import os
# Fixes potential memory fragmentation issues
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Optional: Add this to potentially reduce overall memory usage if needed
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [2]:
# =========================
# 0. INSTALL (run once)
# =========================
!pip install "transformers>=4.41.0" "datasets>=2.19.0" "accelerate>=0.30.0" \
              peft trl bitsandbytes pillow

# If you use Unsloth:
!pip install unsloth



In [3]:
from google.colab import drive
drive.mount('/content/drive')

# My Drive/medgemma_finetune/data
!cp -r "/content/drive/MyDrive/data" "/content/medgemma_finetune/"

Mounted at /content/drive


In [4]:
%cd /content/medgemma_finetune
!python make_instructions.py


/content/medgemma_finetune
[INFO] Dataset: Brain Tumor CT scan Images
  - Class: Healthy
    -> Used 60 images
  - Class: Tumor
    -> Used 60 images
[INFO] Dataset: mammography
  - Class: Malignant
    -> Used 60 images
  - Class: Benign
    -> Used 60 images
[INFO] Dataset: kidney CT
  - Class: Stone
    -> Used 60 images
  - Class: Normal
    -> Used 60 images
  - Class: Cyst
    -> Used 60 images
  - Class: Tumor
    -> Used 60 images
[INFO] Dataset: lung cancer
  - Class: Normal cases
    -> Used 60 images
  - Class: Bengin cases
    -> Used 60 images
  - Class: Malignant cases
    -> Used 60 images
[INFO] Dataset: Brain Tumor MRI images
  - Class: Healthy
    -> Used 60 images
  - Class: Tumor
    -> Used 60 images
[INFO] Dataset: Breast MRI
  - Class: Malignant
    -> Used 60 images
  - Class: Benign
    -> Used 60 images
[INFO] Collected 900 examples total.
[INFO] Train: 765 | Val: 135
[OK] Wrote train_instructions.jsonl and val_instructions.jsonl in /content/medgemma_finetune


In [3]:
import os
from pathlib import Path
from typing import Any

import torch
from datasets import load_dataset
from PIL import Image

from transformers import (
    AutoProcessor,
    AutoModelForImageTextToText,
    BitsAndBytesConfig,
)
from peft import LoraConfig
from trl import SFTConfig, SFTTrainer

In [4]:
PROJECT_ROOT = Path(".")  # current folder
TRAIN_JSONL = "/content/medgemma_finetune/train_instructions.jsonl"
VAL_JSONL   = "/content/medgemma_finetune/val_instructions.jsonl"


MODEL_ID = "unsloth/medgemma-4b-it"   # <-- CHANGE THIS to your Unsloth model id if needed

# Tiny “safer than your GPA” training hyperparams
NUM_EPOCHS = 1
LEARNING_RATE = 1e-5       # you can go 5e-6 if you want to be extra safe
BATCH_SIZE = 4             # adjust by VRAM
GRAD_ACCUM = 2             # effective batch = BATCH_SIZE * GRAD_ACCUM




In [5]:
data = load_dataset(
    "json",
    data_files={
        "train": str(TRAIN_JSONL),
        "validation": str(VAL_JSONL),
    },
)
print(data)

DatasetDict({
    train: Dataset({
        features: ['image_path', 'dataset', 'class_name', 'prompt', 'target'],
        num_rows: 765
    })
    validation: Dataset({
        features: ['image_path', 'dataset', 'class_name', 'prompt', 'target'],
        num_rows: 135
    })
})


In [6]:
# =========================
# 3. FORMAT DATA: image + messages
# =========================

def format_data(example: dict[str, Any]) -> dict[str, Any]:
    """
    Expects each JSONL row to have:
      - image_path: path to image
      - prompt: user prompt text
      - target: JSON string with {label_key, confidence, summary}
    Adds:
      - image: PIL image (RGB)
      - messages: chat-style messages for MedGemma
    """
    img_path = Path(example["image_path"])
    if not img_path.is_absolute():
        img_path = PROJECT_ROOT / img_path

    image = Image.open(img_path).convert("RGB")
    example["image"] = image

    example["messages"] = [
        {
            "role": "user",
            "content": [
                {"type": "image"},
                {"type": "text", "text": example["prompt"]},
            ],
        },
        {
            "role": "assistant",
            "content": [
                {"type": "text", "text": example["target"]},
            ],
        },
    ]
    return example


data = data.map(format_data)
print(data["train"][0])

{'image_path': '/content/medgemma_finetune/data/lung cancer/Bengin cases/Bengin case (37).jpg', 'dataset': 'lung cancer', 'class_name': 'Bengin cases', 'prompt': 'You are an assistant radiologist. Modality: CT chest. Task: Evaluate the lungs for malignant lesions and summarize key findings. Analyze the given medical image and respond ONLY with valid JSON.\n\nThe JSON must contain the keys: "lung_ct_finding", "confidence", "summary".\n', 'target': '{"lung_ct_finding": "Bengin cases", "confidence": 0.74, "summary": "Pulmonary lesion with imaging features more in keeping with a benign process. Recommend review by a qualified specialist and additional imaging if clinically indicated."}', 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=512x512 at 0x79FA5E336A20>, 'messages': [{'content': [{'text': None, 'type': 'image'}, {'text': 'You are an assistant radiologist. Modality: CT chest. Task: Evaluate the lungs for malignant lesions and summarize key findings. Analyze the given m

In [7]:
from huggingface_hub import login
login()


In [8]:
from google.colab import userdata
access_token = userdata.get('HF_TOKEN')

In [ ]:
# =======================================================
# STEP 1: INSPECT MODEL STRUCTURE
# =======================================================
import torch.nn as nn

print("\n--- Model Submodule Names ---")
# List all top-level module names to find the Vision Encoder
for name, module in model.named_children():
    print(f"Top-level Module: {name} (Type: {type(module).__name__})")

print("\n--- Vision Encoder Check ---")
# If 'vision_tower' exists, show its structure
if hasattr(model, 'vision_tower'):
    print("\n'vision_tower' structure:")
    for name, param in model.vision_tower.named_parameters():
        print(f"  {name}: {param.requires_grad}")
        # Only print the first 5 params to avoid huge output
        if name.split('.')[0] == name.split('.')[-1] and len(list(model.vision_tower.named_parameters())) > 5:
            break


--- Model Submodule Names ---


NameError: name 'model' is not defined

In [9]:
import torch
import gc
# ⚠️ Ensure this is one of the first imports in your entire notebook!
from unsloth import FastLanguageModel
from peft import LoraConfig

# =======================================================
# 1. MODEL LOADING
# =======================================================
model_id = "google/medgemma-4b-it"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
model = AutoModelForImageTextToText.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

processor = AutoProcessor.from_pretrained(model_id)
processor.tokenizer.padding_side = "right"

/tmp/ipython-input-3527244008.py:4: UserWarning: WARNING: Unsloth should be imported before trl, transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


config.json:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [10]:
from peft import get_peft_model

In [11]:
# === Step 5: LoRA configuration ===
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.05,
    r=8,
    bias="none",
    target_modules="all-linear",
    task_type="CAUSAL_LM",
    modules_to_save=["lm_head", "embed_tokens"]
)
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

In [12]:
model = get_peft_model(model, peft_config)

In [13]:
model.print_trainable_parameters()

trainable params: 16,394,240 || all params: 4,316,473,712 || trainable%: 0.3798


In [14]:
# =========================
# 6. COLLATE FN (WITH MAX LENGTH FIX)
# =========================

def collate_fn(examples: list[dict[str, Any]]):
    texts = []
    images = []

    for example in examples:
        images.append([example["image"].convert("RGB")])
        texts.append(
            processor.apply_chat_template(
                example["messages"],
                add_generation_prompt=False,
                tokenize=False,
            ).strip()
        )

    batch = processor(
        text=texts,
        images=images,
        return_tensors="pt",
        padding=True,
        truncation=True,  # ADD THIS
        max_length=2048,   # ADD THIS to prevent overflow
    )

    # Create labels
    labels = batch["input_ids"].clone()

    # Mask padding tokens
    labels[labels == processor.tokenizer.pad_token_id] = -100

    # Simple approach: Mask everything except the assistant response
    # This is often better for image-text models
    labels[:, :batch["input_ids"].shape[1] - len(example["messages"][1]["content"][0]["text"])] = -100

    batch["labels"] = labels
    return batch

In [15]:
output_dir = "medgemma-qlora-finetune"
os.makedirs(output_dir, exist_ok=True)

In [16]:
from trl import SFTConfig

args = SFTConfig(
    output_dir=output_dir,
    num_train_epochs=1,
    per_device_train_batch_size=1,        # optimized for low VRAM
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,
    optim="adamw_torch_fused",
    learning_rate=1e-5,
    bf16=False,
    fp16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    push_to_hub=True,
    logging_steps=0.1,
    eval_strategy="no",  # train-only dataset
    report_to="none",
    gradient_checkpointing_kwargs={"use_reentrant": False},
    dataset_kwargs={"skip_prepare_dataset": True},
    remove_unused_columns=False,
    label_names=["labels"],
    dataloader_num_workers=8,
    dataloader_pin_memory=True
)

In [ ]:
# =========================
# 8. TRAINER & TRAIN
# =========================

# You can optionally subsample validation to speed up:
eval_dataset = data["validation"]  # .shuffle().select(range(200))

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=data["train"],
    eval_dataset=eval_dataset,
    processing_class=processor,
    data_collator=collate_fn,
)

print("[INFO] Starting training...")
trainer.train()

[INFO] Starting training...


Step,Training Loss
10,nan
20,nan
30,nan
40,nan
50,nan
60,nan
70,nan
80,nan


In [ ]:
print("[INFO] Starting training...")
trainer.train()

In [ ]:
# Save final adapter
trainer.save_model()  # saves LoRA weights into output_dir
print("[OK] Training complete. LoRA adapter saved in:", args.output_dir)

In [ ]:
save_path = "medgemma-4b-finetuned123"

In [ ]:
trainer.save_model(save_path, safe_serialization=True)
print(f"[OK] Complete model saved to: {save_path}")